This code was made by Juhyeon Lee, Ph.D. candidate (Feb. 7, 2023), Dept. of Brain and Cognitive Engineering, Korea University.

##  ※ "런타임" 클릭  >  "런타임 유형 변경" 클릭 > "GPU" 

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import time
import seaborn as sns
import gc
import os 
import random
from datetime import datetime as dt
from pytz import timezone
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
%matplotlib inline

# Mount Google drive

In [ ]:
# Mount your Google drive to the Google colab notebook 
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/2023_KHBM_WinterSchool')

Mounted at /content/drive


# Load input data

In [ ]:
# !wget 'https://bspl.korea.ac.kr/Research_data/KNU2023/FC_input.npz'
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_npz = drive.CreateFile({'id':'1zFf8EpcZ6Cqsk4K99hUovBNyTFai0fCS'})
data_npz.GetContentFile('FC_input.npz') 


(3200, 19900) (3200, 1)


In [ ]:
fc_data = np.load('FC_input.npz')

X = fc_data['x'] 
X = np.arctanh(X)
X = stats.zscore(X, axis=1)
y = fc_data['y'].astype(np.float_).copy().reshape(-1, 1)

print(X.shape,y.shape)

(3200, 19900) (3200, 1)


In [ ]:
sbjs = fc_data['sbjs']
all_sbjs = np.unique(sbjs)
np.random.shuffle(all_sbjs)

train_sbjs = all_sbjs[0:80]
val_sbjs = all_sbjs[80:90]
test_sbjs = all_sbjs[90:100]

train_sbjs_loc = np.in1d(sbjs,train_sbjs)
valid_sbjs_loc = np.in1d(sbjs,val_sbjs)
test_sbjs_loc = np.in1d(sbjs,test_sbjs)

# Set hyperparameters

In [ ]:
seed = 42

learning_rate = 1e-5
min_lr = 1e-10
lr_patience = 10
momentum = 0.90

l2_param = 5e-5
l1_param_cand = [5e-4,5e-5,5e-6]
param_cand = {"l1_param": l1_param_cand}
param_grid = list(ParameterGrid(param_cand))
param_grid
              
h1 = 512
h2 = 512

dropout_h1 = 0.4
dropout_h2 = 0.4

batch_size = 128
epochs = 300
print_epoch = 10

input_dim = X.shape[1]
output_dim = 1

# Setup Pytorch

In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau 
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

### Set CPU/GPU usage

In [ ]:
num_workers = 2

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 

### Random seed

In [ ]:
def seed_everything(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(seed)

# Define functions

### Define functions for data load

In [ ]:
class GetDataset(Dataset): 
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx): 
        X = torch.from_numpy(self.X[idx]).type(torch.FloatTensor)
        y = torch.from_numpy(self.y[idx]).type(torch.FloatTensor)

        return X, y

### Define functions for training/validaiton/test

In [ ]:
class DNN(nn.Module):
    def __init__(self, h1, h2, dropout_h1, dropout_h2):
        # 2 hidden layers
        super(DNN, self).__init__()
        self.ext_1 = nn.Linear(input_dim, h1)
        self.ext_bn_1 = nn.BatchNorm1d(h1)
        
        self.prd_1 = nn.Linear(h1, h2)
        self.prd_bn_1 = nn.BatchNorm1d(h2)
        self.prd_2 = nn.Linear(h2, output_dim)
        
        self.dropout_h1 = nn.Dropout(p=dropout_h1)
        self.dropout_h2 = nn.Dropout(p=dropout_h2)
        
        self.act_func = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.weights_init()
    
    def forward(self, x):
        x = self.ext_1(x)
        x = self.ext_bn_1(x)
        x = self.act_func(x)
        x = self.dropout_h1(x)
        
        x = self.prd_1(x)
        x = self.prd_bn_1(x)
        x = self.act_func(x)
        x = self.dropout_h2(x)
        x = self.prd_2(x)         
        out = self.sigmoid(x)
        
        return out
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode="fan_in", nonlinearity="relu")
                nn.init.normal_(m.bias, std=0.01)

In [ ]:
def train(model, epoch, train_loader, optimizer, criterion, l1_param, l2_param):
    model.train()
    train_loss = 0
    train_acc = 0
    cost = 0
    correct = 0
    total = 0
    
    for batch_idx, (input, target) in enumerate(train_loader):
        optimizer.zero_grad() 
        input, target = input.to(device), target.to(device)
        pred, target = model(input), target.view(-1, 1)
        
        loss = criterion(pred, target)
        all_linear1_params = torch.cat([x.view(-1) for x in model.ext_1.parameters()])
        all_linear2_params = torch.cat([x.view(-1) for x in model.prd_1.parameters()])
        l1_regularization = l1_param * (torch.norm(all_linear1_params, 1) + torch.norm(all_linear2_params, 1))
        l2_regularization = l2_param * (torch.norm(all_linear1_params, 2) + torch.norm(all_linear2_params, 2))

        cost = loss + l1_regularization + l2_regularization
        cost.backward()
        optimizer.step()
        train_loss += loss.item()
        total += pred.size(0)
                
        crite = torch.FloatTensor([0.5])
        crite = crite.to(device)
        prediction = pred >= crite  # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == target # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도 계산
        correct += accuracy
        
    train_loss = train_loss / total
    train_acc = correct / (batch_idx+1)

    return train_loss, train_acc

In [ ]:
def valid(model, epoch, valid_loader, criterion):
    model.eval()
    valid_loss = 0
    valid_acc = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for input, target in valid_loader:
            input, target = input.to(device), target.to(device)
            pred, target = model(input), target.view(-1, 1)
            loss = criterion(pred, target)
            valid_loss += loss.item()
            total += pred.size(0)
            
            crite = torch.FloatTensor([0.5])
            crite = crite.to(device)
            prediction = pred >= crite # 예측값이 0.5를 넘으면 True로 간주
            correct_prediction = prediction.float() == target # 실제값과 일치하는 경우만 True로 간주
            accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도 계산
            correct += accuracy

    valid_acc = correct
    
    return valid_loss, valid_acc 

In [ ]:
def test(model, epoch, test_loader, criterion):
    model.eval()
    test_loss = 0
    test_acc = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for input, target in test_loader:
            input, target = input.to(device), target.to(device)
            pred, target = model(input), target.view(-1, 1)
            loss = criterion(pred, target)
            test_loss += loss.item()
            total += pred.size(0)
            
            crite = torch.FloatTensor([0.5])
            crite = crite.to(device)
            prediction = pred >= crite  # 예측값이 0.5를 넘으면 True로 간주
            correct_prediction = prediction.float() == target # 실제값과 일치하는 경우만 True로 간주
            accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
            correct += accuracy
            
    test_acc = correct

    return test_loss, test_acc 

### Define function for model training

In [ ]:
def run_train(params, i_model=0, rst_path=None, val_flg=0, print_epoch=10):

    start_time = time.time()

    X_train, y_train = X[train_sbjs_loc,:], y[train_sbjs_loc]
    X_valid, y_valid = X[valid_sbjs_loc,:], y[valid_sbjs_loc]
    X_test, y_test = X[test_sbjs_loc,:], y[test_sbjs_loc]
            
    train_dataset = GetDataset(X_train, y_train)
    valid_dataset = GetDataset(X_valid, y_valid)
    test_dataset = GetDataset(X_test, y_test)
    
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
    valid_loader = DataLoader(
        valid_dataset, batch_size=len(y_valid), pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
    test_loader = DataLoader(
        test_dataset, batch_size=len(y_test), pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
        
    # Assign model 
    model = DNN(h1, h2, dropout_h1, dropout_h2).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, nesterov=True)
    scheduler = ReduceLROnPlateau(
        optimizer, mode="min", patience=lr_patience, min_lr=min_lr
    )
    criterion = nn.functional.binary_cross_entropy
              
    # list to save learning parameters
    train_loss_list = []
    valid_loss_list = []
    train_acc_list = []
    valid_acc_list = []
    test_acc_list = []

    for epoch in range(1, epochs + 1):
        train_loss, train_acc = train(
            model, epoch, train_loader, optimizer, criterion, params['l1_param'], l2_param
        )
        valid_loss, valid_acc = valid(model, epoch, valid_loader, criterion)
        test_loss, test_acc = test(model, epoch, test_loader, criterion)

        scheduler.step(valid_loss)
        lr = optimizer.param_groups[0]['lr']
        
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(valid_acc)
        
        if epoch % print_epoch == 0:
            print("Epoch [{:d}/{:d}]".format(epoch, epochs), end=" ")
            print("Lr: {:.1e}, Train loss: {:.4f}, Valid loss: {:.4f}, Train acc: {:.2f}, Valid acc: {:.2f}"
                  .format(lr, train_loss, valid_loss, train_acc, valid_acc))

            plot_learning_curves(
                rst_path, epochs, train_loss_list, valid_loss_list,  
                train_acc_list, valid_acc_list, i_model, val_flg=1
            )
    
    torch.save(model.state_dict(), 
               rst_path + "/model_" + str(i_model+1) + ".pt")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    acc_list = [train_acc, valid_acc, test_acc]

    tot_time = time.time() - start_time
    print("\nExecution Time for model: {:.2f} mins".format(tot_time / 60))
    
    return train_acc, valid_acc, test_acc

### Define functions for saving results

In [ ]:
def plot_learning_curves(
    rst_path, epochs, train_loss, valid_loss, train_acc, valid_acc, i_model=0, val_flg=0):
    
    sns.set(style="darkgrid", font_scale=2)
    fig, ax = plt.subplots(1, 2, figsize=(18, 10))
    ax = ax.flat
    lw = 4
    last_epoch = epochs
    
    ax[0].plot(train_loss[:last_epoch], label='train loss', lw=lw, color="r")
    if val_flg:
        ax[0].plot(valid_loss[:last_epoch], label='valid loss', lw=lw, color="g")
    ax[0].set_yscale('log')
    ax[0].legend()
    ax[0].set_title("Loss Plot", pad=10)

    ax[1].plot(train_acc[:last_epoch], label='train acc', lw=lw, color="r")
    if val_flg:
        ax[1].plot(valid_acc[:last_epoch], label='valid acc', lw=lw, color="g")
    ax[1].legend()
    ax[1].set_title("Accuracy Plot", pad=10)
    
    fig.tight_layout()
    fig.savefig("{}/Learning_curves_model_{}.png".format(rst_path,i_model+1))
    
    plt.close(fig)

# Create directory & save model info

In [ ]:
nowtime = dt.now(timezone("Asia/Seoul"))

year = str(nowtime.year)
month = '0{}'.format(nowtime.month) if nowtime.month < 10 else str(nowtime.month)
day = '0{}'.format(nowtime.day) if nowtime.day < 10 else str(nowtime.day)
hour = '0{}'.format(nowtime.hour) if nowtime.hour < 10 else str(nowtime.hour)
minute = '0{}'.format(nowtime.minute) if nowtime.minute < 10 else str(nowtime.minute)
sec = '0{}'.format(nowtime.second) if nowtime.second < 10 else str(nowtime.second)

save_path = "result_{}{}{}_{}{}{}".format(year,month,day,hour,minute,sec)
os.makedirs(save_path, exist_ok=True)
rst_path = "{}/train_rst".format(save_path)
os.makedirs(rst_path, exist_ok=True)


f = open(save_path+"/info.txt",'w')
f.write('seed : '+str(seed)+'\n')
f.write('learning_rate : '+str(learning_rate)+'\n')
f.write('min_lr : '+str(min_lr)+'\n')
f.write('lr_patience : '+str(lr_patience)+'\n')
f.write('momentum : '+str(momentum)+'\n')
f.write('l2_param : '+str(l2_param)+'\n')
f.write('dropout_h1 : '+str(dropout_h1)+'\n')
f.write('dropout_h2 : '+str(dropout_h2)+'\n')
f.write('batch_size : '+str(batch_size)+'\n')
f.write('input_dim : '+str(input_dim)+'\n')
f.write('output_dim : '+str(output_dim)+'\n')
f.write('epochs : '+str(epochs)+'\n')
f.close()
np.savez(save_path+'/sbj_shuffle',train_sbjs=train_sbjs,val_sbjs=val_sbjs,test_sbjs=test_sbjs)

print(save_path)

result_20230215_094333


# Start training

In [ ]:
all_acc_list = []
for i_model,params in enumerate(param_grid):

    print("\n<{}> {}".format(i_model+1,params))
    train_acc_model, valid_acc_model, test_acc_model = run_train(
        params, i_model, rst_path, val_flg = 0, print_epoch=print_epoch
    )
    all_acc_list.append([params, train_acc_model, valid_acc_model, test_acc_model])

        
acc_df = pd.DataFrame(np.array(all_acc_list), columns=["params","train_acc", "valid_acc", "test_acc"])
acc_df.to_csv("{}/params_acc.csv".format(rst_path))

all_valid_acc = [item[2] for item in all_acc_list]
selected_model = np.argmax(all_valid_acc)

print("\n<{}> Selected {}, train acc: {:.4f}, valid acc: {:.4f}, test acc: {:.4f}"
      .format(selected_model+1, param_grid[selected_model], all_acc_list[selected_model][1], all_acc_list[selected_model][2], all_acc_list[selected_model][3]))


<1> {'l1_param': 0.0005}
Epoch [10/300] Lr: 1.0e-05, Train loss: 0.0053, Valid loss: 0.5663, Train acc: 0.61, Valid acc: 0.73
Epoch [20/300] Lr: 1.0e-05, Train loss: 0.0045, Valid loss: 0.4761, Train acc: 0.71, Valid acc: 0.81
Epoch [30/300] Lr: 1.0e-05, Train loss: 0.0041, Valid loss: 0.4217, Train acc: 0.73, Valid acc: 0.84
Epoch [40/300] Lr: 1.0e-05, Train loss: 0.0037, Valid loss: 0.3839, Train acc: 0.77, Valid acc: 0.86
Epoch [50/300] Lr: 1.0e-05, Train loss: 0.0036, Valid loss: 0.3559, Train acc: 0.78, Valid acc: 0.87
Epoch [60/300] Lr: 1.0e-05, Train loss: 0.0034, Valid loss: 0.3344, Train acc: 0.80, Valid acc: 0.88
Epoch [70/300] Lr: 1.0e-05, Train loss: 0.0032, Valid loss: 0.3174, Train acc: 0.81, Valid acc: 0.87
Epoch [80/300] Lr: 1.0e-05, Train loss: 0.0029, Valid loss: 0.3025, Train acc: 0.84, Valid acc: 0.88
Epoch [90/300] Lr: 1.0e-05, Train loss: 0.0028, Valid loss: 0.2907, Train acc: 0.85, Valid acc: 0.88
Epoch [100/300] Lr: 1.0e-05, Train loss: 0.0027, Valid loss: 0.28